# <font color='black'>Finding a new location to open up a store based on numbers of competitors in the surrounding area.

Data: FourSq location category & Edmonton neighborhods from Wikipedia

Goal with data:To find a location that isn't already serviced by a competitor. 

Analysis:<font color='red'> Do this part after Habib 

</font> 

#### Doing all the imports I'll probably need & FourSq Url 

In [ ]:
#imports

import pandas as pd
import numpy as np

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install folium
import folium # map rendering library

#foursquare creds

CLIENT_ID = 'SVWMBH0OWIQRNMZEWTZUQJLGHZYUNZRF3TBKC3DP54KX4RTL' # your Foursquare ID
CLIENT_SECRET = 'GBTTM3VQTDMD1RLXKEFVFEJ3WDF4CBQ2I4VPCC5ZIIS5VJG0' # your Foursquare Secret
ACCESS_TOKEN = '22WYCF4PF4XC4JMZ1TKP5MEEBY4USVFON33IFKQEVJHPMYW2' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
# create URL

### Code

In this scenario, I have new entrepreneur that is willing to venture off on their own to start their own recruitment angecy. They've spent years in the industry and now want to be their own boss. They are trying to find the ideal location to start and they are seeking a location that doesn't have too much competition.

Edmonton is surrounded by Highway 216. Our entreprenur wants to be within this circle as that is considered core Edmonton. However, he also does not want to set up shop next to any competitors.

The goal is to find neighborhoods that are don't have a competitor within radius xM. 

I've been tasked with finding an area that they can set up shop in. 
To ensure viability of my data, we will be removing all boroughs & neighborhoods that have no data. This is so our new entrepreneur has data to back his decision. 

In [ ]:
#Setting my location

Location = 'Edmonton, Alberta'

#Creating dataframe for Edmonton Neighborhoods

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
Edmonton=pd.read_html(str(My_table))
#convert table to dataframe
Edmonton=pd.DataFrame(Edmonton[0])
#drop non assigned boroughs to ensure we do not have NaN values & index column from wiki
Edmonton.drop((Edmonton[Edmonton['Borough']=='Not assigned'].index), inplace = True)
Edmonton.drop((Edmonton[Edmonton['Borough']!='Edmonton'].index), inplace = True)
Edmonton.reset_index(drop=True, inplace = True)
#Checking head of my data table

Edmonton.head()

Dataframe of our competitors

In [ ]:
#Creating a dataframe of our competitors using FourSQ

search_query = 'Recruit'
address = 'Edmonton, AB'
geolocator = Nominatim(user_agent="Edm-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(search_query + ' .... OK!')

#Defining the url

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={},{}&oauth_token={}&v={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, LIMIT)
url


In [ ]:
results = requests.get(url).json()
results
# assign relevant part of JSON to competition
competitorsdata = results['response']['venues']

# tranform venues into a dataframe
competitors = json_normalize(competitorsdata)
competitors.head()

In [ ]:
#Filter the dataframe, keep only columns that include venue, name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in competitors.columns if col.startswith('location.')] + ['id']
competitors_data = competitors.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
competitors_data['categories'] = competitors_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
competitors_data.columns = [column.split('.')[-1] for column in competitors_data.columns]
competitors_data = competitors_data.rename(columns={'postalCode':'Postal Code'})
competitors_data

In [ ]:
competitors_data.shape

this section, we will create the map of Edmonton & add in markers for our competitors

In [ ]:
#Getting address for Edmonton
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="Edm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton are {}, {}.'.format(latitude, longitude))

In [ ]:
map_Edmonton = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Edmonton

In [ ]:
#adding competitors to map

for lat, lng, name in zip(competitors_data['lat'],competitors_data['lng'],competitors_data['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=50,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Edmonton) 
map_Edmonton

Now that we have a map of our competitors, we can combine our Edmonton data & competitors data to find the neighborhoods/boroughs in Edmonton that aren't in the 50M radius of each competitor. First, we have to find the difference between the Edmonton table & the Competitors table. That will give us all the locations that aren't in the same area as the competitors. 

In [ ]:
#Using Merge to find Edmonton Boroughs that do not contain competitors
#Also dropping Southindustrial Neighborhood as it has no Lat/Long data. 
#Dropping NaN columns as we do not need them. 
#Renaming Latitude & Longtitugde to lat & lng

outerhoods = Edmonton.merge(competitors_data, on='Postal Code', how='left', indicator=True)
outerhoods = outerhoods[outerhoods.Neighborhood != 'South Industrial']
outerhoods = outerhoods.drop(['categories','address','lat','lng','labeledLatLngs','cc','city','state','country','formattedAddress','crossStreet','id'], axis = 1)
outerhoods = outerhoods.rename(columns={'Latitude':'lat','Longitude':'lng'})
outerhoods['name'].fillna('Free', inplace=True)
outerhoods

In [ ]:
#Adding outerhoods to Edmonton Map

for lat, lng, name in zip(outerhoods['lat'],outerhoods['lng'],outerhoods['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=50,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Edmonton) 
map_Edmonton